In [310]:
# MASTER TRAINING DATASET

# This code gets the pure raw data (with 3 columns added) and converts it into a clean training data set

# The resulting 'master' training dataset contains available results for Olympic Games and World Aquatics Championships between 2012 and 2024

# For the models' evaluation, the training set will be reduced to 2012 to 2023 
# For the Paris Olympic Games predictions, the training set will be kept the same

import pandas as pd
import re

# Reading data from Excel file. The object was called tr_data_master because
# it is the 'master' file containing all the training data, and all other datasets will be created from it
tr_data_master = pd.read_excel('women 100m backstroke raw.xlsx')

# Removing rows with blanks in the 'Rank' column. This ensures that there is only one row per unique swim, instead of two.
tr_data_master = tr_data_master.dropna(subset=['Rank'])

# Formatting the names correctly
def format_name(name):
    # Spliting words and capitalizing properly
    name = re.sub(r'([a-z])([A-Z])', r'\1 \2', name)  # Adding space between lowercase and uppercase
    name = ' '.join(word.capitalize() for word in name.split())  
    return name

tr_data_master['Swimmer name'] = tr_data_master['Swimmer name'].apply(format_name)

# Cleaning the 'Swimming time' column. All times are displayed in seconds.
def clean_time(time):
    time_str = str(time)
    # Extracting minutes and seconds if the time is in format 'm:ss.xx' (the case of events like the 100m breaststroke)
    if ':' in time_str:
        match = re.search(r'(\d+):(\d+\.\d+)', time_str)
        if match:
            minutes = int(match.group(1))
            seconds = float(match.group(2))
            total_seconds = minutes * 60 + seconds
            return '{:.2f}'.format(total_seconds)
    # Extracting seconds if time is in format 'ss.xx' (the case of events like the 50m freestyle)
    else:
        match = re.search(r'(\d+\.\d+)', time_str)
        if match:
            return match.group(1)
    return time_str 

tr_data_master['Swimming time'] = tr_data_master['Swimming time'].apply(clean_time)

# Converting 'Swimming time' to string and replacing commas with periods
tr_data_master['Swimming time'] = tr_data_master['Swimming time'].astype(str).str.replace(',', '.')
# Making sure that each 'Swimming time' value has two decimal places
tr_data_master['Swimming time'] = tr_data_master['Swimming time'].apply(lambda x: '{:.2f}'.format(float(x)))

# Deleting rows with 'Reaction time' equal to '-'
tr_data_master = tr_data_master[tr_data_master['Reaction time'] != '-']
# Converting 'Reaction time' to string and replacing commas with periods
tr_data_master['Reaction time'] = tr_data_master['Reaction time'].astype(str).str.replace(',', '.')
# Making sure that each 'Reaction time' value has two decimal places
tr_data_master['Reaction time'] = tr_data_master['Reaction time'].apply(lambda x: '{:.2f}'.format(float(x)))

# Replacing '-' with 0 in the 'Behind time' column
tr_data_master['Behind time'] = tr_data_master['Behind time'].replace('-', '0')
# Removing leading '+' from values
tr_data_master['Behind time'] = tr_data_master['Behind time'].str.lstrip('+')
# Making sure that 'Behind time' is in the correct numeric format
tr_data_master['Behind time'] = tr_data_master['Behind time'].apply(lambda x: '{:.2f}'.format(float(x)))

# Filtering out swimmers who will be 30 or older in 2024
# It is unlikely that swimmers of this age participate in an international competition in 2024
tr_data_master['Age_in_2024'] = tr_data_master['Age'] + (2024 - tr_data_master['Year of competition'])
tr_data_master = tr_data_master[tr_data_master['Age_in_2024'] < 30]

# Dropping the temporary 'Age_in_2024' column as it is no longer needed
tr_data_master = tr_data_master.drop(columns=['Age_in_2024'])

# Filtering out swimmers who only have one row. This way we can have information about each swimmer
# both in the training and test sets, avoiding data leakage.
swimmer_counts = tr_data_master['Swimmer name'].value_counts()
swimmers_with_multiple_rows = swimmer_counts[swimmer_counts > 1].index
tr_data_master = tr_data_master[tr_data_master['Swimmer name'].isin(swimmers_with_multiple_rows)]

In [311]:
# Testing for multicollinearity

from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# Ensuring the relevant columns are numeric
tr_data_master['Behind time'] = pd.to_numeric(tr_data_master['Behind time'], errors='coerce')
tr_data_master['Rank'] = pd.to_numeric(tr_data_master['Rank'], errors='coerce')

# Dropping rows with NaN values (which may have been introduced by the conversion)
tr_data_master = tr_data_master.dropna(subset=['Behind time', 'Rank'])

# Selecting the relevant variables
variables = tr_data_master[['Behind time', 'Rank']]

# Adding a constant (intercept) to the dataset
variables_with_const = sm.add_constant(variables)

# Calculating VIF
vif_data = pd.DataFrame()
vif_data['Variable'] = variables_with_const.columns
vif_data['VIF'] = [variance_inflation_factor(variables_with_const.values, i) for i in range(variables_with_const.shape[1])]

print(vif_data)

      Variable       VIF
0        const  2.327812
1  Behind time  3.851133
2         Rank  3.851133


In [312]:
# TRAINING DATASET FOR PARIS PREDICTIONS
# This includes swimming results from 2012 to 2024, available before the Paris Olympic Games

# Sorting tr_data_master by 'Swimmer name', 'Year of competition' and 'Swimming time'
tr_data_master_sorted = tr_data_master.sort_values(by=['Swimmer name', 'Year of competition', 'Swimming time'], ascending=[True, False, True])

# Grouping by 'Swimmer name' and excluding the first row from each group. These first rows will be used in the test set.
tr_data_Paris = tr_data_master_sorted.groupby('Swimmer name').apply(lambda x: x.iloc[1:]).reset_index(drop=True)

In [313]:
# TRAINING DATASET FOR MODELS' EVALUATION

# Including only the years 2012 to 2023. 2024 data will be used in the test set.
tr_data_ev = tr_data_Paris[(tr_data_Paris['Year of competition'] >= 2012) & (tr_data_Paris['Year of competition'] <= 2023)]

In [314]:
# TEST DATASET FOR MODELS' EVALUATION

# This test set is created by combining fields from 'unique_swimmers' and 'Doha_results'
# Only unique swimmers are kept, prioriting the most recent swim first and the fastest time second

# Including only the years 2012 to 2023 (before the Doha World Aquatics Championships in 2024) 
step1_test_data_ev = tr_data_master_sorted[(tr_data_master_sorted['Year of competition'] >= 2012) & (tr_data_master_sorted['Year of competition'] <= 2023)]

# Dropping duplicates based on 'Swimmer name', 
# We only keep the row with the most recent swim (and fastest time)
# Some values from these rows will be used to create the test set for the models' evaluation
unique_swimmers = step1_test_data_ev.drop_duplicates(subset='Swimmer name', keep='first')

# Defining an object containing swim results for the 2024 Doha World Aquatics Championships
Doha_results = tr_data_master[(tr_data_master['Competition name'] == 'Doha World Aquatics Championships')]
# Sorting Doha_results by 'Swimmer name', 'Swimming time' and 'Rank'
Doha_results = Doha_results.sort_values(by=['Swimmer name', 'Swimming time', 'Rank'], ascending=[True, True, True])
# Dropping duplicates based on 'Swimmer name', keeping the row with the fastest time (and best rank)
# The fastest times will be y_test, which will be compared to the predictions to determine how accurate these were
Doha_results = Doha_results.drop_duplicates(subset='Swimmer name', keep='first')

# Now both the 'unique_swimmers' and the 'Doha_results' datasets have no duplicate swimmers

# Next, we use 'unique_swimmers' and 'Doha_results' to create a new dataframe

test_data_ev = pd.DataFrame()

# Populating the dataframe with the relevant columns from 'Doha_results' and 'unique_swimmers_tr_ev'
test_data_ev['Swimmer name'] = Doha_results['Swimmer name']
test_data_ev['Swimming time'] = Doha_results['Swimming time'] # future y_test
test_data_ev['Competition name'] = 'Doha World Aquatics Championships'
test_data_ev['Year of competition'] = 2024
test_data_ev['Age'] = Doha_results['Age']
test_data_ev['Country'] = Doha_results['Country']

# Merging to get 'Rank', 'Lane', 'Reaction Time', 'Behind time', 'Points' and 'Phase' from 'unique_swimmers'
test_data_ev = test_data_ev.merge(unique_swimmers[['Swimmer name', 'Rank', 'Lane', 'Reaction time', 'Behind time', 'Points', 'Phase']], on='Swimmer name', how='inner')

# Reordering the columns
test_data_ev = test_data_ev[['Swimmer name', 'Swimming time', 'Points', 'Rank', 'Lane', 'Country', 'Age', 'Reaction time', 'Behind time', 'Competition name', 'Year of competition', 'Phase']]

In [315]:
# CODE FOR MAKING PREDICTIONS FOR THE DOHA WORLD AQUATICS CHAMPIONSHIPS AND COMPARING THESE PREDICTIONS TO THE ACTUAL RESULTS

import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Using the required columns for training
X_train = tr_data_ev[['Age', 'Competition name', 'Year of competition', 'Country', 'Rank', 'Behind time', 'Lane', 'Reaction time']]
y_train = tr_data_ev['Swimming time']

# Using the required columns for testing
X_test = test_data_ev[['Age', 'Competition name', 'Year of competition', 'Country', 'Rank', 'Behind time', 'Lane', 'Reaction time']]
y_test = test_data_ev['Swimming time']

# Defining a pipeline for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Competition name', 'Country']),
        ('num', SimpleImputer(strategy='mean'), ['Age', 'Year of competition', 'Rank', 'Behind time', 'Lane', 'Reaction time'])
    ]
)

# Training a LINEAR REGRESSION model
lr_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Defining the hyperparameter grid for Linear Regression
param_grid_lr = {
    'regressor__fit_intercept': [True, False]
}

# Performing GridSearchCV to find the best hyperparameters for Linear Regression
lr_grid_search = GridSearchCV(lr_model, param_grid_lr, cv=5, scoring='neg_mean_squared_error')
lr_grid_search.fit(X_train, y_train)

# Extracting the best Linear Regression model with tuned hyperparameters
best_lr_model = lr_grid_search.best_estimator_

# Training a DECISION TREE model
decision_tree_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=42))
])

# Defining the hyperparameter grid for Decision Tree
param_grid_decision_tree = {
    'regressor__max_depth': [None, 5, 10, 20],
    'regressor__min_samples_split': [2, 5, 10]
}

# Performing GridSearchCV to find the best hyperparameters for Decision Tree
decision_tree_grid_search = GridSearchCV(decision_tree_model, param_grid_decision_tree, cv=5, scoring='neg_mean_squared_error')
decision_tree_grid_search.fit(X_train, y_train)

# Extracting the best Decision Tree model with tuned hyperparameters
best_decision_tree_model = decision_tree_grid_search.best_estimator_

# Training a RANDOM FOREST model
rf_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Defining the hyperparameter grid for Random Forest
param_grid_rf = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [2, 5, 10]
}

# Performing GridSearchCV to find the best hyperparameters for Random Forest
rf_grid_search = GridSearchCV(rf_model, param_grid_rf, cv=5, scoring='neg_mean_squared_error')
rf_grid_search.fit(X_train, y_train)

# Extracting the best Random Forest model with tuned hyperparameters
best_rf_model = rf_grid_search.best_estimator_

# Training a SUPPORT VECTOR MACHINE model
svm_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR())
])

# Defining the hyperparameter grid for SVM
param_grid_svm = {
    'regressor__C': [0.1, 1, 10, 100],
    'regressor__gamma': ['scale', 'auto', 0.1, 1, 10]
}

# Performing GridSearchCV to find the best hyperparameters for SVM
svm_grid_search = GridSearchCV(svm_model, param_grid_svm, cv=5, scoring='neg_mean_squared_error')
svm_grid_search.fit(X_train, y_train)

# Extracting the best SVM model with tuned hyperparameters
best_svm_model = svm_grid_search.best_estimator_

# Making predictions on the test set
lr_predictions = best_lr_model.predict(X_test)
decision_tree_predictions = best_decision_tree_model.predict(X_test)
rf_predictions = best_rf_model.predict(X_test)
svm_predictions = best_svm_model.predict(X_test)

# Calculating R-squared for each model on the test set
r2_lr = r2_score(y_test, lr_predictions)
r2_decision_tree = r2_score(y_test, decision_tree_predictions)
r2_rf = r2_score(y_test, rf_predictions)
r2_svm = r2_score(y_test, svm_predictions)

# Calculating Mean Absolute Error for each model on the test set
mae_lr = mean_absolute_error(y_test, lr_predictions)
mae_decision_tree = mean_absolute_error(y_test, decision_tree_predictions)
mae_rf = mean_absolute_error(y_test, rf_predictions)
mae_svm = mean_absolute_error(y_test, svm_predictions)

# Calculating Mean Squared Error for each model on the test set
mse_lr = mean_squared_error(y_test, lr_predictions)
mse_decision_tree = mean_squared_error(y_test, decision_tree_predictions)
mse_rf = mean_squared_error(y_test, rf_predictions)
mse_svm = mean_squared_error(y_test, svm_predictions)

# Calculating Root Mean Squared Error for each model on the test set
rmse_lr = np.sqrt(mse_lr)
rmse_decision_tree = np.sqrt(mse_decision_tree)
rmse_rf = np.sqrt(mse_rf)
rmse_svm = np.sqrt(mse_svm)

print('\nModel\t\tR-squared\tMean Absolute Error\tMean Squared Error\tRoot Mean Squared Error')
print('Linear Regression\t{:.4f}\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}'.format(r2_lr, mae_lr, mse_lr, rmse_lr))
print('Decision Trees\t\t{:.4f}\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}'.format(r2_decision_tree, mae_decision_tree, mse_decision_tree, rmse_decision_tree))
print('Random Forest\t\t{:.4f}\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}'.format(r2_rf, mae_rf, mse_rf, rmse_rf))
print('SVM\t\t\t{:.4f}\t\t{:.4f}\t\t\t{:.4f}\t\t\t{:.4f}'.format(r2_svm, mae_svm, mse_svm, rmse_svm))


Model		R-squared	Mean Absolute Error	Mean Squared Error	Root Mean Squared Error
Linear Regression	0.9248		0.8282			1.2767			1.1299
Decision Trees		0.9390		0.7248			1.0356			1.0177
Random Forest		0.9451		0.6853			0.9334			0.9661
SVM			0.9553		0.6290			0.7594			0.8714


In [316]:
# Getting the feature importances for each model (for Doha)

from sklearn.inspection import permutation_importance

# Setting pandas to display a certain number of rows
pd.set_option('display.max_rows', None)

# Linear Regression feature importances
ohe = best_lr_model.named_steps['preprocessor'].named_transformers_['cat']
feature_names = ohe.get_feature_names_out(['Competition name', 'Country']).tolist() + ['Age', 'Year of competition', 'Rank', 'Behind time', 'Lane', 'Reaction time']
lr_coefficients = best_lr_model.named_steps['regressor'].coef_
lr_feature_importances = pd.Series(np.abs(lr_coefficients), index=feature_names).sort_values(ascending=False)

print("\nLinear Regression Feature Importances:")
print(lr_feature_importances.apply(lambda x: f"{x:.4f}"))

# Decision Tree feature importances
dt_feature_importances = pd.Series(best_decision_tree_model.named_steps['regressor'].feature_importances_, index=feature_names).sort_values(ascending=False)

print("\nDecision Tree Feature Importances:")
print(dt_feature_importances.apply(lambda x: f"{x:.4f}"))

# Random Forest feature importances
rf_feature_importances = pd.Series(best_rf_model.named_steps['regressor'].feature_importances_, index=feature_names).sort_values(ascending=False)

print("\nRandom Forest Feature Importances:")
print(rf_feature_importances.apply(lambda x: f"{x:.4f}"))

# SVM feature importances (if the kernel is linear)
if isinstance(best_svm_model.named_steps['regressor'], SVR) and best_svm_model.named_steps['regressor'].kernel == 'linear':
    svm_coefficients = best_svm_model.named_steps['regressor'].coef_
    svm_feature_importances = pd.Series(np.abs(svm_coefficients.flatten()), index=feature_names).sort_values(ascending=False)
    print("\nSVM Feature Importances (Linear Kernel):")
    print(svm_feature_importances.apply(lambda x: f"{x:.4f}"))
else:
    print("\nSVM Feature importances are not directly available for non-linear kernels.")

# Getting feature importances for non-linear SVM using permutation importance
result = permutation_importance(best_svm_model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)

# Converting importances to Series and aligning with variable names
svm_feature_importances = pd.Series(result.importances_mean, index=X_test.columns).sort_values(ascending=False)

print("\nPermutation Feature Importances for SVM:")
print(svm_feature_importances.apply(lambda x: f"{x:.4f}")) 


Linear Regression Feature Importances:
Country_MAW                                                  1.0966
Country_MDV                                                  1.0755
Country_SWZ                                                  0.9220
Behind time                                                  0.8980
Competition name_Barcelona World Aquatics Championships      0.7247
Competition name_Tokyo Olympic Games                         0.7232
Country_TKM                                                  0.4825
Competition name_Budapest World Aquatics Championships II    0.3944
Country_MGL                                                  0.3625
Competition name_Rio Olympic Games                           0.3448
Country_CAM                                                  0.3425
Competition name_Fukuoka World Aquatics Championships        0.3372
Country_GRN                                                  0.2499
Country_INA                                                  0.2306
Country_

In [317]:
# TEST DATASET FOR PARIS PREDICTIONS

# Keeping unique swimmers, prioritizing most recent swim first and fastest time second
# As a reminder, the training data for Paris was created earlier (tr_data_Paris)

# 'tr_data_master_sorted' was defined earlier. It sorts 'tr_data_master' by 'Swimmer name', 'Year of competition', and 'Swimming time'
# We drop duplicates based on 'Swimmer name', keeping the row with the most recent swim (and fastest time)
test_data_Paris = tr_data_master_sorted.drop_duplicates(subset='Swimmer name', keep='first').copy()

# Updating the 'Competition name', 'Year of competition', and 'Age' columns
test_data_Paris['Competition name'] = 'Paris Olympic Games'
test_data_Paris['Age'] = test_data_Paris['Age'] + (2024 - test_data_Paris['Year of competition'])
test_data_Paris['Year of competition'] = 2024 

# Reordering the columns as specified
test_data_Paris = test_data_Paris[['Swimmer name', 'Swimming time', 'Points', 'Rank', 'Lane', 'Country', 'Age', 'Reaction time', 'Behind time', 'Competition name', 'Year of competition', 'Phase']]
# It is worth noting that, since the 'Swimming time' column from test_data_Paris will never be used, it does not matter which values it contains

In [318]:
# Setting up the models for the Paris Olympics predictions

# Splitting data into features (X) and target (y)
X_train = tr_data_Paris[['Age', 'Competition name', 'Year of competition', 'Country', 'Lane', 'Rank', 'Behind time', 'Reaction time']]
y_train = tr_data_Paris['Swimming time']

X_test = test_data_Paris[['Age', 'Competition name', 'Year of competition', 'Country', 'Lane', 'Rank', 'Behind time', 'Reaction time']]
y_test = test_data_Paris['Swimming time']

# Defining a pipeline for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Competition name', 'Country']),
        ('num', SimpleImputer(strategy='mean'), ['Age', 'Year of competition', 'Lane', 'Rank', 'Behind time', 'Reaction time'])
    ]
)

# Training a LINEAR REGRESSION model
lr_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Defining the hyperparameter grid for Linear Regression
param_grid_lr = {
    'regressor__fit_intercept': [True, False]
}

# Performing GridSearchCV to find the best hyperparameters for Linear Regression
lr_grid_search = GridSearchCV(lr_model, param_grid_lr, cv=5, scoring='neg_mean_squared_error')
lr_grid_search.fit(X_train, y_train)

# Extracting the best Linear Regression model with tuned hyperparameters
best_lr_model = lr_grid_search.best_estimator_

# Training a DECISION TREE model
decision_tree_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=42))
])

# Defining the hyperparameter grid for Decision Tree
param_grid_decision_tree = {
    'regressor__max_depth': [None, 5, 10, 20],
    'regressor__min_samples_split': [2, 5, 10]
}

# Performing GridSearchCV to find the best hyperparameters for Decision Tree
decision_tree_grid_search = GridSearchCV(decision_tree_model, param_grid_decision_tree, cv=5, scoring='neg_mean_squared_error')
decision_tree_grid_search.fit(X_train, y_train)

# Extracting the best Decision Tree model with tuned hyperparameters
best_decision_tree_model = decision_tree_grid_search.best_estimator_

# Training a RANDOM FOREST model
rf_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Defining the hyperparameter grid for Random Forest
param_grid_rf = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [2, 5, 10]
}

# Performing GridSearchCV to find the best hyperparameters for Random Forest
rf_grid_search = GridSearchCV(rf_model, param_grid_rf, cv=5, scoring='neg_mean_squared_error')
rf_grid_search.fit(X_train, y_train)

# Extracting the best Random Forest model with tuned hyperparameters
best_rf_model = rf_grid_search.best_estimator_

# Training a SUPPORT VECTOR MACHINE model
svm_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR())
])

# Defining the hyperparameter grid for SVM
param_grid_svm = {
    'regressor__C': [0.1, 1, 10, 100],
    'regressor__gamma': ['scale', 'auto', 0.1, 1, 10]
}

# Performing GridSearchCV to find the best hyperparameters for SVM
svm_grid_search = GridSearchCV(svm_model, param_grid_svm, cv=5, scoring='neg_mean_squared_error')
svm_grid_search.fit(X_train, y_train)

# Extracting the best SVM model with tuned hyperparameters
best_svm_model = svm_grid_search.best_estimator_

In [319]:
# PREDICTING PARIS OLYMPICS SWIMMING TIMES, FINALISTS, AND OVERALL RANKING

# Setting pandas to display a certain number of rows
pd.set_option('display.max_rows', None)

# Making predictions on the test set
lr_predictions = best_lr_model.predict(X_test)
decision_tree_predictions = best_decision_tree_model.predict(X_test)
rf_predictions = best_rf_model.predict(X_test)
svm_predictions = best_svm_model.predict(X_test)

# Storing the predictions in dataframes
lr_predictions_df = pd.DataFrame({
    'Swimmer name': test_data_Paris['Swimmer name'],
    'Country': test_data_Paris['Country'],
    'LR_Prediction': lr_predictions
})

decision_tree_predictions_df = pd.DataFrame({
    'Swimmer name': test_data_Paris['Swimmer name'],
    'Country': test_data_Paris['Country'],
    'Decision_Tree_Prediction': decision_tree_predictions
})

rf_predictions_df = pd.DataFrame({
    'Swimmer name': test_data_Paris['Swimmer name'],
    'Country': test_data_Paris['Country'],
    'Random_Forest_Prediction': rf_predictions
})

svm_predictions_df = pd.DataFrame({
    'Swimmer name': test_data_Paris['Swimmer name'],
    'Country': test_data_Paris['Country'],
    'SVM_Prediction': svm_predictions
})

# Function to filter the top 2 athletes per country (this is to comply with an Olympic Games rule)
def filter_two_athletes(df, time_col):
    # Sorting by 'Country' and 'Swimming time'
    df_sorted = df.sort_values(by=['Country', time_col], ascending=[True, True])
    # Selecting the fastest 2 athletes per country
    top_athletes_per_country = df_sorted.groupby('Country').head(2)
    return top_athletes_per_country

# Applying the filtering function to each model's predictions. Results are sorted by country in alphabetical order.
top_lr = filter_two_athletes(lr_predictions_df, 'LR_Prediction').drop(columns=['Country'])
top_decision_tree = filter_two_athletes(decision_tree_predictions_df, 'Decision_Tree_Prediction').drop(columns=['Country'])
top_rf = filter_two_athletes(rf_predictions_df, 'Random_Forest_Prediction').drop(columns=['Country'])
top_svm = filter_two_athletes(svm_predictions_df, 'SVM_Prediction').drop(columns=['Country'])

# Sorting the top athletes by predicted time
top_lr = top_lr.sort_values(by='LR_Prediction')
top_decision_tree = top_decision_tree.sort_values(by='Decision_Tree_Prediction')
top_rf = top_rf.sort_values(by='Random_Forest_Prediction')
top_svm = top_svm.sort_values(by='SVM_Prediction')

# Function to convert seconds to minute:seconds format, if seconds >= 60 (for better readability)
def conditional_convert_seconds_to_min_sec(seconds):
    if seconds >= 60:
        minutes = int(seconds // 60)
        remaining_seconds = seconds % 60
        return f"{minutes}:{remaining_seconds:05.2f}"
    else:
        return f"{seconds:.2f}" 

# Converting the sorted times to minute:seconds format, for better readability
def formatted_time(df, col_name):
    df[col_name] = df[col_name].apply(conditional_convert_seconds_to_min_sec)
    return df

top_lr = formatted_time(top_lr, 'LR_Prediction')
top_decision_tree = formatted_time(top_decision_tree, 'Decision_Tree_Prediction')
top_rf = formatted_time(top_rf, 'Random_Forest_Prediction')
top_svm = formatted_time(top_svm, 'SVM_Prediction')

# Printing the fastest swimmers and their times for each model
print('Top swimmers and their times for Linear Regression:')
print(top_lr[['Swimmer name', 'LR_Prediction']])

print('\nTop swimmers and their times for Decision Trees:')
print(top_decision_tree[['Swimmer name', 'Decision_Tree_Prediction']])

print('\nTop swimmers and their times for Random Forest:')
print(top_rf[['Swimmer name', 'Random_Forest_Prediction']])

print('\nTop swimmers and their times for SVM:')
print(top_svm[['Swimmer name', 'SVM_Prediction']])

# Selecting the top 8 unique swimmers for each model
top_8_lr = top_lr.head(8)
top_8_decision_tree = top_decision_tree.head(8)
top_8_rf = top_rf.head(8)
top_8_svm = top_svm.head(8)

# Printing the top 8 unique swimmers and their times for each model
print('\nTop 8 swimmers and their times for Linear Regression:')
print(top_8_lr[['Swimmer name', 'LR_Prediction']])

print('\nTop 8 swimmers and their times for Decision Trees:')
print(top_8_decision_tree[['Swimmer name', 'Decision_Tree_Prediction']])

print('\nTop 8 swimmers and their times for Random Forest:')
print(top_8_rf[['Swimmer name', 'Random_Forest_Prediction']])

print('\nTop 8 swimmers and their times for SVM:')
print(top_8_svm[['Swimmer name', 'SVM_Prediction']])

Top swimmers and their times for Linear Regression:
                 Swimmer name LR_Prediction
1356            Claire Curzan         58.68
1186           Kaylee Mckeown         58.70
766            Minna Atherton         58.82
1188              Regan Smith         58.87
1208              Kylie Masse         59.27
1414          Kathleen Dawson         59.28
1360              Ingrid Wilm         59.44
1416     Carmen Weiler Sastre         59.44
1210           Pauline Mahieu         59.57
1420          Maaike De Waard         59.59
760             Natsumi Sakai         59.63
776             Daria Vaskina         59.67
316            Lauren Quigley         59.72
1214               Letian Wan         59.84
592          Daria K Ustinova         59.98
1248               Xueer Wang       1:00.05
1110              Emma Terebo       1:00.06
952        Anastasia Gorbenko       1:00.07
1382           Kira Toussaint       1:00.10
1424            Hanna Rosvall       1:00.14
956           Mariia Kam